In [1]:
from autoray import numpy as np
import symmray as sr
import torch
import numpy
import quimb.tensor as qtn
import autoray as ar
from autoray import do
from quimb.tensor.tensor_core import  *
from quimb.tensor.tensor_core import bonds, tags_to_oset, rand_uuid
from quimb.tensor.tensor_2d import Rotator2D, pairwise

def from_netket_config_to_quimb_config(netket_configs):
    def func(netket_config):
        """Translate netket spin-1/2 config to tensor network product state config"""
        total_sites = len(netket_config)//2
        spin_up = netket_config[:total_sites]
        spin_down = netket_config[total_sites:]
        sum_spin = spin_up + spin_down
        quimb_config = np.zeros(total_sites, dtype=int)
        for i in range(total_sites):
            if sum_spin[i] == 0:
                quimb_config[i] = 0
            if sum_spin[i] == 2:
                quimb_config[i] = 3
            if sum_spin[i] == 1:
                if spin_down[i] == 1:
                    quimb_config[i] = 1
                else:
                    quimb_config[i] = 2
        return quimb_config
    if len(netket_configs.shape) == 1:
        return func(netket_configs)
    else:
        # batched
        return np.array([func(netket_config) for netket_config in netket_configs])

def from_quimb_config_to_netket_config(quimb_config):
    """Translate tensor network product state config to netket spin-1/2 config"""
    total_sites = len(quimb_config)
    spin_up = np.zeros(total_sites, dtype=int)
    spin_down = np.zeros(total_sites, dtype=int)
    for i in range(total_sites):
        if quimb_config[i] == 0:
            spin_up[i] = 0
            spin_down[i] = 0
        if quimb_config[i] == 1:
            spin_up[i] = 0
            spin_down[i] = 1
        if quimb_config[i] == 2:
            spin_up[i] = 1
            spin_down[i] = 0
        if quimb_config[i] == 3:
            spin_up[i] = 1
            spin_down[i] = 1
    return np.concatenate((spin_up, spin_down))

def get_spinful_parity_map():
    return {0:0, 1:1, 2:1, 3:0}

def get_spinful_charge_map():
    return {0:0, 1:1, 2:1, 3:2}

def from_spinful_ind_to_charge(config):
    charge_map = get_spinful_charge_map()
    return np.array([charge_map[n] for n in config])

/home/sijingdu/anaconda3/envs/vmc_torch/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/sijingdu/anaconda3/envs/vmc_torch/lib/python3.9/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [2]:
class fPEPS(qtn.PEPS):
    def __init__(self, arrays, *, shape="urdlp", tags=None, site_ind_id="k{},{}", site_tag_id="I{},{}", x_tag_id="X{}", y_tag_id="Y{}", **tn_opts):
        super().__init__(arrays, shape=shape, tags=tags, site_ind_id=site_ind_id, site_tag_id=site_tag_id, x_tag_id=x_tag_id, y_tag_id=y_tag_id, **tn_opts)
        self.symmetry = self.arrays[0].symmetry
        self.spinless = True if self.phys_dim() == 2 else False
    
    def product_bra_state(self, config):
        product_tn = qtn.TensorNetwork()
        backend = self.tensors[0].data.backend
        device = config.device
        dtype = eval(backend+'.'+self.tensors[0].data.dtype)

        if self.spinless:
            index_map = {0: 0, 1: 1}
            array_map = {
                0: do('array',[1.0,],like=config,dtype=dtype,device=device), 
                1: do('array',[1.0,],like=config,dtype=dtype,device=device)
            }
        else:
            if self.symmetry == 'Z2':
                index_map = {0:0, 1:1, 2:1, 3:0}
                array_map = {
                    0: do('array',[1.0, 0.0],like=config,dtype=dtype,device=device), 
                    1: do('array',[1.0, 0.0],like=config,dtype=dtype,device=device), 
                    2: do('array',[0.0, 1.0],like=config,dtype=dtype,device=device), 
                    3: do('array',[0.0, 1.0],like=config,dtype=dtype,device=device)
                }
            elif self.symmetry == 'U1':
                index_map = {0:0, 1:1, 2:1, 3:2}
                array_map = {
                    0: do('array',[1.0,],like=config,dtype=dtype,device=device), 
                    1: do('array',[1.0, 0.0],like=config,dtype=dtype,device=device), 
                    2: do('array',[0.0, 1.0],like=config,dtype=dtype,device=device), 
                    3: do('array',[1.0,],like=config,dtype=dtype,device=device)
                }

        for n, site in zip(config, self.sites):
            p_ind = self.site_ind_id.format(*site)
            p_tag = self.site_tag_id.format(*site)
            tid = self.sites.index(site)

            n_charge = index_map[int(n)]
            n_array = array_map[int(n)]

            oddpos = None
            if not self.spinless:
                # assert self.symmetry == 'U1', "Only U1 symmetry is supported for spinful fermions for now."
                if int(n) == 1:
                    oddpos = (3*tid+1)*(-1)#**reverse
                elif int(n) == 2:
                    oddpos = (3*tid+2)*(-1)#**reverse
                elif int(n) == 3:
                    # oddpos = ((3*tid+1)*(-1)**reverse, (3*tid+2)*(-1)**reverse)
                    oddpos = None
            else:
                oddpos = (3*tid+1)*(-1)

            tsr_data = sr.FermionicArray.from_blocks(
                blocks={(n_charge,):n_array}, 
                duals=(True,),
                symmetry=self.symmetry, 
                charge=n_charge, 
                oddpos=oddpos
            )
            tsr = qtn.Tensor(data=tsr_data, inds=(p_ind,),tags=(p_tag, 'bra'))
            product_tn |= tsr

        return product_tn
    
    # NOTE: don't use @classmethod here, as we need to access the specific instance attributes
    def get_amp(self, config, inplace=False, conj=True):
        """Get the amplitude of a configuration in a PEPS."""
        peps = self if inplace else self.copy()
        product_state = self.product_bra_state(config).conj() if conj else self.product_bra_state(config)
        
        amp = peps|product_state # ---T---<---|n>
        
        for site in peps.sites:
            site_tag = peps.site_tag_id.format(*site)
            amp.contract_(tags=site_tag)

        amp.view_as_(
            qtn.PEPS,
            site_ind_id="k{},{}",
            site_tag_id="I{},{}",
            x_tag_id="X{}",
            y_tag_id="Y{}",
            Lx=peps.Lx,
            Ly=peps.Ly,
        )
        return amp
    
def generate_random_fpeps(Lx, Ly, D, seed, symmetry='Z2', Nf=0, cyclic=False, spinless=True):
    """Generate a random spinless/spinful fermionic square PEPS of shape (Lx, Ly)."""

    assert symmetry == 'Z2' or symmetry == 'U1', "Only Z2 and U1 symmetries are supported."
    
    edges = qtn.edges_2d_square(Lx, Ly, cyclic=cyclic)
    site_info = sr.utils.parse_edges_to_site_info(
        edges,
        D,
        phys_dim=2 if spinless else 4,
        site_ind_id="k{},{}",
        site_tag_id="I{},{}",
    )

    peps = qtn.TensorNetwork()
    rng = np.random.default_rng(seed)
    charge_config = np.zeros(Lx*Ly, dtype=int)

    # generate a random binary string with Nf ones in it
    if symmetry == 'U1':
        if spinless:
            charge_config[:Nf] = 1
            rng.shuffle(charge_config)
        else:
            charge_config_netket = from_quimb_config_to_netket_config(charge_config)
            charge_config_netket[:Nf] = 1
            rng.shuffle(charge_config_netket)
            charge_config = from_spinful_ind_to_charge(from_netket_config_to_quimb_config(charge_config_netket))

    elif symmetry == 'Z2':
        parity_config = charge_config

    for site, info in sorted(site_info.items()):
        tid = site[0] * Ly + site[1]
        # bond index charge distribution
        block_indices = [
            sr.BlockIndex({0: d // 2, 1: d // 2}, dual=dual)
            for d, dual in zip(info["shape"][:-1], info["duals"][:-1])
        ]
        # physical index
        p = info['shape'][-1]
        if symmetry == 'Z2':
            block_indices.append(
                sr.BlockIndex({0: p // 2, 1: p // 2}, dual=info["duals"][-1])
            )
        elif symmetry == 'U1':
            if spinless:
                block_indices.append(
                sr.BlockIndex({0: p // 2, 1: p // 2}, dual=info["duals"][-1])
            )
            else:
                block_indices.append(
                    sr.BlockIndex({0: p // 4, 1: p // 2, 2: p // 4}, dual=info["duals"][-1])
                )
        
        # random fermionic array
        if symmetry == 'Z2':
            data = sr.Z2FermionicArray.random(
                block_indices,
                charge=1 if parity_config[tid] else 0,
                seed=rng,
                oddpos=3*tid,
            )
        elif symmetry == 'U1':
            data = sr.U1FermionicArray.random(
                block_indices,
                charge=int(charge_config[tid]),
                seed=rng,
                oddpos=3*tid,
            )

        peps |= qtn.Tensor(
            data=data,
            inds=info["inds"],
            tags=info["tags"],
        )

    # required to view general TN as an actual PEPS
    for i, j in site_info:
        peps[f"I{i},{j}"].add_tag([f"X{i}", f"Y{j}"])

    peps.view_as_(
        fPEPS,
        site_ind_id="k{},{}",
        site_tag_id="I{},{}",
        x_tag_id="X{}",
        y_tag_id="Y{}",
        Lx=Lx,
        Ly=Ly,
    )
    peps = peps.copy() # set symmetry during initialization
    assert peps.spinless == spinless

    return peps, charge_config

In [3]:
class fMPS(qtn.MatrixProductState):
    def __init__(
        self,
        arrays,
        *,
        sites=None,
        L=None,
        shape="lrp",
        tags=None,
        site_ind_id="k{}",
        site_tag_id="I{}",
        **tn_opts,
    ):
        super().__init__(arrays, sites=sites, L=L, shape=shape, tags=tags, site_ind_id=site_ind_id, site_tag_id=site_tag_id, **tn_opts)

        self.symmetry = self.arrays[0].symmetry
        self.spinless = True if self.phys_dim() == 2 else False
    
    def product_bra_state(self, config):
        product_tn = qtn.TensorNetwork()
        backend = self.tensors[0].data.backend
        device = config.device
        dtype = eval(backend+'.'+self.tensors[0].data.dtype)

        if self.spinless:
            index_map = {0: 0, 1: 1}
            array_map = {
                0: do('array',[1.0,],like=config,dtype=dtype,device=device), 
                1: do('array',[1.0,],like=config,dtype=dtype,device=device)
            }
        else:
            if self.symmetry == 'Z2':
                index_map = {0:0, 1:1, 2:1, 3:0}
                array_map = {
                    0: do('array',[1.0, 0.0],like=config,dtype=dtype,device=device), 
                    1: do('array',[1.0, 0.0],like=config,dtype=dtype,device=device), 
                    2: do('array',[0.0, 1.0],like=config,dtype=dtype,device=device), 
                    3: do('array',[0.0, 1.0],like=config,dtype=dtype,device=device)
                }
            elif self.symmetry == 'U1':
                index_map = {0:0, 1:1, 2:1, 3:2}
                array_map = {
                    0: do('array',[1.0,],like=config,dtype=dtype,device=device), 
                    1: do('array',[1.0, 0.0],like=config,dtype=dtype,device=device), 
                    2: do('array',[0.0, 1.0],like=config,dtype=dtype,device=device), 
                    3: do('array',[1.0,],like=config,dtype=dtype,device=device)
                }

        for n, site in zip(config, self.sites):
            p_ind = self.site_ind_id.format(site)
            p_tag = self.site_tag_id.format(site)
            tid = self.sites.index(site)

            n_charge = index_map[int(n)]
            n_array = array_map[int(n)]

            oddpos = None
            if not self.spinless:
                # assert self.symmetry == 'U1', "Only U1 symmetry is supported for spinful fermions for now."
                if int(n) == 1:
                    oddpos = (3*tid+1)*(-1)#**reverse
                elif int(n) == 2:
                    oddpos = (3*tid+2)*(-1)#**reverse
                elif int(n) == 3:
                    # oddpos = ((3*tid+1)*(-1)**reverse, (3*tid+2)*(-1)**reverse)
                    oddpos = None
            else:
                oddpos = (3*tid+1)*(-1)

            tsr_data = sr.FermionicArray.from_blocks(
                blocks={(n_charge,):n_array}, 
                duals=(True,),
                symmetry=self.symmetry, 
                charge=n_charge, 
                oddpos=oddpos
            )
            tsr = qtn.Tensor(data=tsr_data, inds=(p_ind,),tags=(p_tag, 'bra'))
            product_tn |= tsr

        return product_tn
    
    # NOTE: don't use @classmethod here, as we need to access the specific instance attributes
    def get_amp(self, config, inplace=False, conj=True):
        """Get the amplitude of a configuration in a PEPS."""
        mps = self if inplace else self.copy()
        product_state = self.product_bra_state(config).conj() if conj else self.product_bra_state(config)
        
        amp = mps|product_state # ---T---<---|n>
        
        for site in mps.sites:
            site_tag = mps.site_tag_id.format(site)
            amp.contract_(tags=site_tag)

        amp.view_as_(
            qtn.MatrixProductState,
            site_ind_id="k{}",
            site_tag_id="I{}",
            L=mps.L,
            cyclic=mps.cyclic,
        )
        return amp

In [22]:


def generate_random_fmps(L, D, seed, symmetry='Z2', Nf=0, cyclic=False, spinless=True):
    """Generate a random spinless/spinful fermionic MPS of length L."""
    assert symmetry == 'Z2' or symmetry == 'U1', "Only Z2 and U1 symmetries are supported."

    edges = qtn.edges_1d_chain(L, cyclic=cyclic)
    site_info = sr.utils.parse_edges_to_site_info(
        edges,
        D,
        phys_dim=2 if spinless else 4,
        site_ind_id="k{}",
        site_tag_id="I{}",
    )

    mps = qtn.TensorNetwork()
    rng = np.random.default_rng(seed)
    charge_config = np.zeros(L, dtype=int)

    # generate a random binary string with Nf ones in it
    if symmetry == 'U1':
        if spinless:
            charge_config[:Nf] = 1
            rng.shuffle(charge_config)
        else:
            charge_config_netket = from_quimb_config_to_netket_config(charge_config)
            charge_config_netket[:Nf] = 1
            rng.shuffle(charge_config_netket)
            charge_config = from_spinful_ind_to_charge(from_netket_config_to_quimb_config(charge_config_netket))

    elif symmetry == 'Z2':
        parity_config = charge_config

    for site, info in sorted(site_info.items()):
        tid = site
        # bond index charge distribution
        block_indices = [
            sr.BlockIndex({0: d // 2, 1: d // 2}, dual=dual)
            for d, dual in zip(info["shape"][:-1], info["duals"][:-1])
        ]
        # physical index
        p = info['shape'][-1]
        if symmetry == 'Z2':
            block_indices.append(
                sr.BlockIndex({0: p // 2, 1: p // 2}, dual=info["duals"][-1])
            )
        elif symmetry == 'U1':
            if spinless:
                block_indices.append(
                sr.BlockIndex({0: p // 2, 1: p // 2}, dual=info["duals"][-1])
            )
            else:
                block_indices.append(
                    sr.BlockIndex({0: p // 4, 1: p // 2, 2: p // 4}, dual=info["duals"][-1])
                )
        
        # random fermionic array
        if symmetry == 'Z2':
            data = sr.Z2FermionicArray.random(
                block_indices,
                charge=1 if parity_config[tid] else 0,
                seed=rng,
                oddpos=3*tid,
            )
        elif symmetry == 'U1':
            data = sr.U1FermionicArray.random(
                block_indices,
                charge=int(charge_config[tid]),
                seed=rng,
                oddpos=3*tid,
            )
        
        mps |= qtn.Tensor(
            data=data,
            inds=info["inds"],
            tags=info["tags"],
        )

    # required to view general TN as an actual PEPS

    mps.view_as_(
        fMPS,
        L=L,
        site_ind_id="k{}",
        site_tag_id="I{}",
        cyclic=cyclic,
    )
    mps = mps.copy() # set symmetry during initialization
    return mps, charge_config


In [23]:
# fMPS spinful
L = 4
Nf = L
D = 4
symmetry = 'U1'
spinless = False
cyclic = False
seed = 2
# SU in quimb
fmps, charge_config = generate_random_fmps(L, D, seed, symmetry, Nf, cyclic, spinless)
edges = qtn.edges_1d_chain(L, cyclic=False)
site_info = sr.utils.parse_edges_to_site_info(
    edges,
    D,
    phys_dim=2 if spinless else 4,
    site_ind_id="k{}",
    site_tag_id="I{}",
)

t = 1.0
U = 8.0
mu = 0.0

terms = {
    (sitea, siteb): sr.fermi_hubbard_local_array(
        t=t, U=U, mu=mu,
        symmetry=symmetry,
        coordinations=(
            site_info[sitea]['coordination'],
            site_info[siteb]['coordination'],
        ),
    ).fuse((0, 1), (2, 3))
    for (sitea, siteb) in fmps.gen_bond_coos()
}
ham = qtn.LocalHam1D(L, terms, cyclic=False)

su = qtn.SimpleUpdateGen(fmps, ham, compute_energy_per_site=True, D=D, gate_opts={'cutoff':1e-12})

# cluster energies may not be accuracte yet
su.evolve(50, tau=0.3)
su.evolve(50, tau=0.1)
su.evolve(50, tau=0.03)
su.evolve(100, tau=0.01)
su.evolve(100, tau=0.003)
su.evolve(100, tau=0.001)

n=450, tau=0.0010, energy~-0.263498: 100%|##########| 100/100 [00:00<00:00, 136.22it/s]


In [ ]:
import os
os.environ["NUMBA_NUM_THREADS"] = "20"

import netket as nk
import netket.experimental as nkx
import netket.nn as nknn

from math import pi

from netket.experimental.operator.fermion import destroy as c
from netket.experimental.operator.fermion import create as cdag
from netket.experimental.operator.fermion import number as nc

from vmc_torch.fermion_utils import generate_random_fmps
import quimb.tensor as qtn
import symmray as sr
import pickle

# Define the lattice shape
L = 4 # length of the chain
spinless = False
# graph = nk.graph.Square(L)
graph = nk.graph.Chain(L, pbc=False)
N = graph.n_nodes

# Define the fermion filling and the Hilbert space
N_f = int(L)
n_fermions_per_spin = (N_f//2, N_f//2)
hi = nkx.hilbert.SpinOrbitalFermions(N, s=1/2, n_fermions_per_spin=n_fermions_per_spin)

# Define the Hubbard Hamiltonian
t = 1.0
U = 8.0
mu = 0.0

H = 0.0
for (i, j) in graph.edges(): # Definition of the Hubbard Hamiltonian
    for spin in (1,-1):
        H -= t * (cdag(hi,i,spin) * c(hi,j,spin) + cdag(hi,j,spin) * c(hi,i,spin))
for i in graph.nodes():
    H += U * nc(hi,i,+1) * nc(hi,i,-1)


# Exact diagonalization of the Hamiltonian for benchmark
sp_h = H.to_sparse() # Convert the Hamiltonian to a sparse matrix
from scipy.sparse.linalg import eigsh
eig_vals, eig_vecs = eigsh(sp_h, k=2, which="SA")
E_gs = eig_vals[0]
print("Exact ground state energy per site:", E_gs/N)